# Tasks
piplineは便利ですが、抽象的でわかりにくいです。
そこでタスクの理解を深めるために、piplineを使わないでモデルを使用してみましょう。
ここを理解することによって、ファインチューニングの実装が容易になります。

## Sentiment Analysis：ポジネガ判定
事業評価などに特化したポジネガ判定

- Model: [jarvisx17/japanese-sentiment-analysis](https://huggingface.co/jarvisx17/japanese-sentiment-analysis)
- Dataset: [chABSA](https://www.kaggle.com/datasets/takahirokubo0/chabsa)
    - アスペクトベース感情分析(ABSA)は、文の極性やその根拠を理解するのに重要である。しかし、そのような重要なデータセットであるにもかかわらず、利用可能な言語やドメインが限られています。本論文では、日本語のABSAデータセット（chABSA-dataset）を提供する。chABSA-datasetは企業分析領域に基づいて構築されており、我々の知る限り、日本語のABSAデータセットとしては初めてのものである。


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("jarvisx17/japanese-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("jarvisx17/japanese-sentiment-analysis")

classes = ["positive", "negative"]

sequence_0 = "飼養戸数が全ての畜種で減少する中で、1戸当たりの飼養頭羽数は、いずれの畜種でも増加傾向となっています"
sequence_1 = "国産小麦は、収穫期が降雨の時期に重なること、赤カビ等の病害が発生しやすく、また、外国産小麦に比べてタンパク含有量のばらつきが大きいこと等、品質上の課題もあります。"
sequence_2 = "新型コロナウイルス感染症からの経済活動の回復・在庫の確保などを受け、令和3年の緑茶の輸出額は204億円と、過去最高額を記録しました。"

for sequence in [sequence_0, sequence_1, sequence_2]:
    input = tokenizer(sequence, return_tensors="pt")

    classification_logits = model(**input).logits

    results = torch.softmax(classification_logits, dim=1).tolist()[0]
    
    print(sequence)
    for i in range(len(classes)):
        print(f"{classes[i]}: {int(round(results[i] * 100))}%")


飼養戸数が全ての畜種で減少する中で、1戸当たりの飼養頭羽数は、いずれの畜種でも増加傾向となっています
positive: 100%
negative: 0%
国産小麦は、収穫期が降雨の時期に重なること、赤カビ等の病害が発生しやすく、また、外国産小麦に比べてタンパク含有量のばらつきが大きいこと等、品質上の課題もあります。
positive: 100%
negative: 0%
新型コロナウイルス感染症からの経済活動の回復・在庫の確保などを受け、令和3年の緑茶の輸出額は204億円と、過去最高額を記録しました。
positive: 0%
negative: 100%



### やっていること
1. チェックポイント名からトークナイザーとモデルをインスタンス化します。
2. モデルはBERTモデルとして識別され、チェックポイントに格納された重みをロードします。
3. 判定対象の文を定義します。
4. このシーケンスをモデルに通して、2つの利用可能なクラスのうちの1つに分類されるようにする。0（positive）、1（negative）。
5. クラスに対する確率を得るために、結果のソフトマックスを計算する。
6. 結果を表示する。

## Question Answering：テキストから回答の抽出

In [15]:
from transformers import AutoTokenizer, RobertaForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("tsmatz/roberta_qa_japanese")
model = RobertaForQuestionAnswering.from_pretrained("tsmatz/roberta_qa_japanese")

text = "太郎は昨日はカレーを食べました。今日はお腹があまり減っていません。\
    街をぶらぶらと歩いていると、立ち飲み屋が目に入りました。太郎はこのお店が前からちょっと気になっていました。\
    しかし、太郎は入ることなく、そのお店の前を通り抜けていきます。太郎は家に帰ると、チュールを飼い主からもらい、食べました。"

questions = ["太郎の昨日のご飯は？", "太郎の今日のご飯は？", '太郎は人間ですか？']

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"人の質問: {question}")
    print(f"AIの回答: {answer}")

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

人の質問: 太郎の昨日のご飯は？
AIの回答: カレー
人の質問: 太郎の今日のご飯は？
AIの回答: カレー
人の質問: "太郎は人間ですか？
AIの回答: チュール


### やっていること
1. チェックポイント名からトークナイザーとモデルをインスタンス化します。モデルは、BERT モデルとして識別され、チェックポイントに格納されている重みでロードされます。
2. テキストといくつかの質問を定義する。
3. 質問を繰り返し、テキストと現在の質問から、正しいモデル固有のセパレータ、トークンタイプID、アテンションマスクを使ってシーケンスを構築します。
4. このシーケンスをモデルに渡します。これにより、開始位置と終了位置の両方について、シーケンストークン（質問とテキスト）全体にわたるスコアの範囲が出力される。
5. 結果のソフトマックスを計算し、トークンの確率を得る。
6. 識別された開始と終了の値からトークンを取得し、それらのトークンを文字列に変換する。
7. 結果を表示する。

## Masked LM：穴埋め


In [13]:
from transformers import AutoTokenizer, BertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
model = BertForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

sequence = f"「ドラえもん」というアニメは、日本で人気のアニメーション作品です。このアニメは、小学生の主人公・のび太が、\n【{tokenizer.mask_token}】ができるという能力を持つロボット、ドラえもんと出会い、さまざまな冒険をするという話を描いています。"

inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


「ドラえもん」というアニメは、日本で人気のアニメーション作品です。このアニメは、小学生の主人公・のび太が、
【変身】ができるという能力を持つロボット、ドラえもんと出会い、さまざまな冒険をするという話を描いています。
「ドラえもん」というアニメは、日本で人気のアニメーション作品です。このアニメは、小学生の主人公・のび太が、
【仕事】ができるという能力を持つロボット、ドラえもんと出会い、さまざまな冒険をするという話を描いています。
「ドラえもん」というアニメは、日本で人気のアニメーション作品です。このアニメは、小学生の主人公・のび太が、
【飛行】ができるという能力を持つロボット、ドラえもんと出会い、さまざまな冒険をするという話を描いています。
「ドラえもん」というアニメは、日本で人気のアニメーション作品です。このアニメは、小学生の主人公・のび太が、
【遊び】ができるという能力を持つロボット、ドラえもんと出会い、さまざまな冒険をするという話を描いています。
「ドラえもん」というアニメは、日本で人気のアニメーション作品です。このアニメは、小学生の主人公・のび太が、
【冒険】ができるという能力を持つロボット、ドラえもんと出会い、さまざまな冒険をするという話を描いています。


### やっていること
1. チェックポイント名からトークナイザーとモデルをインスタンス化します。モデルはBERTモデルとして識別され、チェックポイントに格納された重みでロードされます。
2. tokenizer.mask_tokenを単語の代わりに置き、マスクされたトークンを持つシーケンスを定義する。
3. そのシーケンスをIDのリストにエンコードし、そのリストにおけるマスクされたトークンの位置を見つけます。
4. マスクトークンのインデックスで予測値を取得する。このテンソルは語彙と同じサイズであり、値は各トークンに帰着するスコアである。このモデルはその文脈で可能性が高いと思われるトークンに対して高いスコアを与える。
5. PyTorchのtopkメソッドやTensorFlowのtop_kメソッドを用いて上位5つのトークンを取得する。
6. マスクトークンをトークンで置き換え、結果を表示する。

## テキスト生成


In [38]:
from transformers import XLNetTokenizer, XLNetLMHeadModel, T5Tokenizer, GPT2LMHeadModel, top_k_top_p_filtering
from fugashi import Tagger 
import torch
from torch import nn

tokenizer_gpt2 = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer_gpt2.do_lower_case = True  # due to some bug of tokenizer config loading
model_gpt2 = GPT2LMHeadModel.from_pretrained("rinna/japanese-gpt2-small")

tokenizer_xlnet = XLNetTokenizer.from_pretrained("hajime9652/xlnet-japanese")
model_xlnet = XLNetLMHeadModel.from_pretrained("hajime9652/xlnet-japanese")
tagger = Tagger('-Owakati') 

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = "パソコンを立ち上げてGmailを開くと、とあるホテルからメールが届いていた。メールに中にあった冬景色の中に佇む凛とした佇まいの旅館の写真を見て、あの時の旅行のことを思い出した。\
    あの日は金沢への出張だった。出張とは言っても予定はクライアントと食事をするだけだった。時間に余裕のあった私は街からちょっと外れた隠れ家のような旅館を予約していた。"

prompt = "東京から新幹線に乗ると、"
inputs = tokenizer_xlnet.encode(tagger.parse(PADDING_TEXT + prompt), add_special_tokens=False, return_tensors="pt")


prompt_length = len(tokenizer_xlnet.decode(inputs[0]))
outputs = model_xlnet.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer_xlnet.decode(outputs[0])[prompt_length + 1 :]

print(generated)

inputs = tokenizer_gpt2(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

prompt_length = len(tokenizer_gpt2.decode(inputs[0]))
outputs = model_gpt2.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer_gpt2.decode(outputs[0])[prompt_length + 1 :]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


東京から新幹線に乗ると、この ホテル の 街 に 住ん て い た 筆者 。 私 を 探す の は 、 「 そんな お家 は いかにも 憧れ てす ね 。 その 土地 に ある もの か ある 。 たから 、 とう て も あり ませ ん てし た か 。 そのほか の 旅行 に関する アンケート に 聞く 。 その 結果 、 筆者 は ホテル の なか から ちょうと 自分 の 不動産 を 購入 し て い た こと は あり ます か? 自分 は 好き たっ た からす 。 「 月 は 、 とても 幸せ に なり まし た ね 」 の コメント を くれ た 。
東京から新幹線に乗ると、沢駅までの片道1時間弱の短い時間で片道20～30分の場所に温泉がある。ホテルの温泉は2つ用意されていた。 宿泊の翌日、ビジネスホテルがお決まりに。そんなわけで宿泊先の温泉宿の玄関で「お客さまありがとうございました」という挨拶を交わした。そして1階のロビーへ出ると、私の手が私の肩を掴んでくる。そしてロビーに案内されて「ありがとうございました」と。こうして旅館でマッサージを受けつつ、3人部屋に案内される。 2人で食事をした時に、テーブルに腰かけてもらった。そう、ベッドに入った瞬間に全身が震えていた。 ホテルではマッサージがメインだったが、マッサージを受けることは初めてだった。足の裏を洗ってもらえると、気持ちがリラックス


### やっていること
1. チェックポイント名からトークナイザーとモデルをインスタンス化します。
2. モデルはGPT2モデル / XLNetモデルとして識別され、チェックポイントに格納された重みでロードされます。
3. 生成したいテキストの前文を定義します。（PADDING TEXTはXLNetには必要な処理です。）
4. model.generateでテキストを生成しています。
5. 結果を表示しています。

## Named Entity Recognition：固有表現認識
名前付き固有表現認識（NER）は、トークンをあるクラスに従って分類するタスクで、例えば、トークンを人、組織、場所として識別することができます。

In [34]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("tsmatz/xlm-roberta-ner-japanese")
model = AutoModelForTokenClassification.from_pretrained("tsmatz/xlm-roberta-ner-japanese")


sequence = (
    "太郎は4月の陽気の良い日に、鈴をつけて熊本県の阿蘇山に登った"
    "女性不足を止めるには、どこで働くかより、いつ働くか"
)

inputs = tokenizer(sequence, return_tensors="pt")
tokens = inputs.tokens()

outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

Downloading:   0%|          | 0.00/451 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

### やっていること
1. チェックポイント名からトークナイザーとモデルをインスタンス化します。
2. モデルはBERTモデルとして識別され、チェックポイントに格納された重みでロードされます。
3. 既知のエンティティでシーケンスを定義します。
4. 単語をトークンに分割し、予測にマッピングできるようにします。まず、シーケンスを完全にエンコード、デコードし、特殊トークンを含む文字列を残すという、ちょっとしたハックを使います。
5. その文字列をIDにエンコードします（特殊トークンは自動的に追加されます）。
6. 入力をモデルに渡し、最初の出力を得ることで予測値を取得します。この結果、各トークンについて9つの可能なクラスに対する分布が得られます。各トークンについて最も可能性の高いクラスを取得するために、argmaxを取ります。
7. 各トークンとその予測値を束ねて表示します。

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("tsmatz/mt5_summarize_japanese")
model = AutoModelForSeq2SeqLM.from_pretrained("tsmatz/mt5_summarize_japanese")

ARTICLE = "ロジスティック写像（ロジスティックしゃぞう、英語: logistic map）とは、xn+1 = axn(1 − xn) という2次関数の差分方程式（漸化式）で定められた離散力学系である。\
    ロジスティックマップ[1][2][3]や離散型ロジスティック方程式（英語: discrete logistic equation）[4][5][6]とも呼ばれる。\
    単純な2次関数の式でありながら、驚くような複雑な振る舞いを生み出すことで知られる。\
    ロジスティック写像の a はパラメータと呼ばれる定数、x が変数で、適当に a の値を決め、最初の x0 を決めて計算すると、x0, x1, x2, … という数列が得られる。\
    この数列を力学系分野では軌道と呼び、軌道は a にどのような値を与えるかによって変化する。\
    パラメータ a を変化させると、ロジスティック写像の軌道は、一つの値へ落ち着いたり、いくつかの値を周期的に繰り返したり、カオスと呼ばれる非周期的変動を示したりと様々に変化する。\
    ロジスティック写像を生物の個体数を表すモデルとして見る立場からは、変数 xn は1世代目、2世代目…というように世代ごとに表した個体数を意味しており、\
    ロジスティック写像とは現在の個体数 xn から次の世代の個体数 xn+1 を計算する式である。\
    生物個体数モデルとしてのロジスティック写像は、ある生物の個体数がある環境中に生息し、さらにその環境と外部との間で個体の移出入がないような状況を想定しており、\
    xn は正確には個体数そのものではなく、その環境中に存在できる最大個体数に対する割合を意味する。\
    微分方程式で個体数をモデリングするロジスティック方程式の離散化からもロジスティック写像は導出でき、「ロジスティック写像」という名もそのことに由来する。\
    2次関数の力学系としての研究は20世紀初頭からあったが、1970年代、特に数理生物学者ロバート・メイの研究によってロジスティック写像は広く知られるようになった。\
    メイ以外にも、スタニスワフ・ウラムとジョン・フォン・ノイマン、ペッカ・ミュルバーク（フィンランド語版）、\
    オレクサンドル・シャルコフスキー（ウクライナ語版）、ニコラス・メトロポリス（英語版）ら、ミッチェル・ファイゲンバウムなどがロジスティック写像の振る舞い解明に関わる仕事を成している。"

inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### やっていること
1. チェックポイント名からトークナイザーとモデルをインスタンス化します。要約は通常、Bart や T5 のようなエンコーダ・デコーダモデルを用いて行われます。
2. 要約されるべき記事を定義します。
4. T5特有の接頭辞 "summarize: "が必要です。
5. PreTrainedModel.generate() メソッドを用いて要約を生成します。

## 翻訳

In [39]:
from transformers import MBartForConditionalGeneration, MBartTokenizer

tokenizer = MBartTokenizer.from_pretrained("ken11/mbart-ja-en")
model = MBartForConditionalGeneration.from_pretrained("ken11/mbart-ja-en")

inputs = tokenizer(
    "謹んで新春をお祝い申し上げます。\
    旧年中は大変お世話になり、誠にありがとうございました。○○様におかれましては、今年のお正月も、ご家族と共に楽しくお過ごしのことと存じます。\
    今年は開発部に異動となり、心機一転、あらためて業務に努めていく所存でございます。今年も変わらぬご指導ご鞭撻のほど何卒よろしくお願い申し上げます。\
    本年も御家族の皆様のご多幸を心からお祈り申し上げます。",
    return_tensors="pt",
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

translated_tokens = model.generate(**inputs, decoder_start_token_id=tokenizer.lang_code_to_id["en_XX"], early_stopping=True, max_length=48)
pred = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
print(pred)

Downloading:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

### やっていること
1. チェックポイント名からトークナイザーとモデルをインスタンス化します。要約は通常、Bart や T5 のようなエンコーダ・デコーダモデルを用いて行われます。
2. 要約されるべき記事を定義します。
3. T5特有のプレフィックス "translate Japanese to English: " を追加します。
4. PreTrainedModel.generate()メソッドを使用して翻訳を実行します。

# 演習
- 上記のタスク全てで、オリジナルの入力データを使うことで、挙動を確認してください。
- どういった入力で、どういった出力になるかを観察してください。たとえばテキスト生成では、同じ文が繰り返し出力されるなどは既知の問題です。